## <font size='5' color='red'>Content</font>

Hey fellow Kagglers,this is a one strange competition among the all the competitions that has been lauched in kaggle till date.So,I dedicate this notebook to share what I undertood about the competition and **Abstract Reasoning**.
![](https://miro.medium.com/max/2081/1*KVomuoRF3Io41g4Zhjy2iw.jpeg)

To understand why abstract reasoning is critical for general intelligence, consider Archimedes’ famous “Eureka!” moment: by noticing that the volume of an object is equivalent to the volume of water that the object displaces, he understood volume at a conceptual level, and was therefore able to reason about the volume of other irregularly shaped objects.The ability to relate two abstract concepts also allowed Albert Einstein to formulate the basics of the theory of relativity as he reasoned that an equivalence relation exists between an observer falling in uniform acceleration and an observer in a uniform gravitational field. Abstract reasoning has long been used as an example that separates human cognition from artificial intelligence(AI)

We would like AI to have similar capabilities. While current systems can defeat world champions in complicated strategic games, they often struggle on other apparently simple tasks, especially when an abstract concept needs to be discovered and reapplied in a new setting. For example, if specifically trained to only count triangles, then even our best AI systems can still fail to count squares, or any other previously unencountered object.So,I hope you now understand the basic difference between our problem and the traditional artificial intelligence problems that we are familiar with.Now,we will move onto exploring this competition..

In [ ]:
import numpy as np
import pandas as pd

import os
import json
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
    
from pathlib import Path


### <font size='4' color='red'>Abstract reasoning dataset</font>

Abstract Reasoning Corpus (ARC) comprises a training set and an evaluation set. The training set features 400 tasks,while the evaluation set features 400 tasks.  The evaluation set is further split into a publicevaluation set (400 tasks) and a private evaluation set (100 tasks). All tasks are unique, and the set of test tasks and the set of training tasks are disjoint.Let's check this first..

In [ ]:
data_path = Path('/kaggle/input/abstraction-and-reasoning-challenge/')
training_path = data_path / 'training'
evaluation_path = data_path / 'evaluation'
test_path = data_path / 'test'

training_tasks = sorted(os.listdir(training_path))
evaluation_tasks = sorted(os.listdir(evaluation_path))
test_tasks = sorted(os.listdir(test_path))
print("Number of examples in training corpus is ",len(training_tasks))
print("Number of examples in evaluation corpus is ",len(evaluation_tasks))
print("Number of examples in testing corpus is ",len(test_tasks))


Okay,now each task consists of a small number of demonstration examples (3 on average), and a small number of test examples (generally 1, although it may be 2 or 3 in rare cases). Each example consists of an “input grid” and an “output grid”.  Each “grid” is a literal grid of symbols (each symbol is typically visualized via a unique color). Thereare 10 unique symbols (or colors).  A grid can be any height or width between 1x1 and 30x30, inclusive (the median height is 9 and the median width is 10). Let's understand this by checking our training corpus.

In [ ]:

cmap = colors.ListedColormap(
    ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
     '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)

def plot_task(task):
    n = len(task["train"]) + len(task["test"])
    fig, axs = plt.subplots(2, n, figsize=(4*n,8), dpi=50)
    plt.subplots_adjust(wspace=0, hspace=0)
    fig_num = 0
    for i, t in enumerate(task["train"]):
        t_in, t_out = np.array(t["input"]), np.array(t["output"])
        axs[0][fig_num].imshow(t_in, cmap=cmap, norm=norm)
        axs[0][fig_num].set_title(f'Train-{i} in')
        axs[0][fig_num].set_yticks(list(range(t_in.shape[0])))
        axs[0][fig_num].set_xticks(list(range(t_in.shape[1])))
        axs[1][fig_num].imshow(t_out, cmap=cmap, norm=norm)
        axs[1][fig_num].set_title(f'Train-{i} out')
        axs[1][fig_num].set_yticks(list(range(t_out.shape[0])))
        axs[1][fig_num].set_xticks(list(range(t_out.shape[1])))
        fig_num += 1
    for i, t in enumerate(task["test"]):
        t_in, t_out = np.array(t["input"]), np.array(t["output"])
        axs[0][fig_num].imshow(t_in, cmap=cmap, norm=norm)
        axs[0][fig_num].set_title(f'Test-{i} in')
        axs[0][fig_num].set_yticks(list(range(t_in.shape[0])))
        axs[0][fig_num].set_xticks(list(range(t_in.shape[1])))
        axs[1][fig_num].imshow(t_out, cmap=cmap, norm=norm)
        axs[1][fig_num].set_title(f'Test-{i} out')
        axs[1][fig_num].set_yticks(list(range(t_out.shape[0])))
        axs[1][fig_num].set_xticks(list(range(t_out.shape[1])))
        fig_num += 1
    
    plt.tight_layout()
    plt.show()
    

In [ ]:
for i, json_path in enumerate(training_tasks[:3]):
    
    task_file = str(training_path / json_path)

    with open(task_file, 'r') as f:
        task = json.load(f)

    print(f"{i:03d}", task_file)
    plot_task(task)

Now you can see that for each task there are n number of training pairs and 1 test pair.We need to build an algorithm to that is trained on these n training pairs and can predict the output pattern for the test input.Also you can observe that the grid size is not same for all tasks,our algoithm should first be able to figure out the output grid size given some training pairs.
So now you know your objective we will move onto submission section.

## <font size='3' color='red'>prediction format</font>

For each test task,you can use 3 trials.If anyone of these trials exactly match the correct output matrix,the answer is regarded as correct.




In [ ]:

submission = pd.read_csv(data_path / 'sample_submission.csv', index_col='output_id')
display(submission.head())


The output_id is the id of the task, followed by the index of the test input that you should use to make your prediction. The output is the predicted output of the corresponding test input, reformatted into a string representation.Let's check how this is represented..

In [ ]:
print("Number of submissions for each test task is" ,len(submission['output'][0].strip().split(' ')))

Let's try and print an image correponding to one output

In [ ]:
def show_output(outputid):
    
    fig, axs = plt.subplots(1, 3, figsize=(15,15))
    l=0
    for sub in submission.loc[outputid]['output'].strip().split(' '):
        out=[]
        for i in sub.split('|')[1:-1]:
                x=list(map(int,list(i)))
                out.append(x)

        axs[l].imshow(out,cmap=cmap)
        axs[l].axis('off')
        l=l+1


In [ ]:
show_output('009d5c81_0')

Now you can see the three predicted ouput patterns for test task 1 . For each test task there are 3 trails like this and if any of these exactly match the expected output your submission is regarded as correct.

## <font size='4' color='red'>Please leave an upvote if you feel this was helpful<font size='3' color='blue'>   
##    Thank you</font></font>

### References
- https://deepmind.com/blog/article/measuring-abstract-reasoning
- https://arxiv.org/pdf/1911.01547.pdf
- https://towardsdatascience.com/are-neural-networks-capable-of-abstract-reasoning-lets-use-an-iq-test-to-prove-it-1d1ff1929fef